In [109]:
import os
import re

patterns = {
    r"\bany2json\b": "archery",
    r"\bany2Json\b": "archery",
    r"\bAny2json\b": "Archery",
    r"\bAny2Json\b": "Archery",
}

ignores = [".git", "target"]

includes = [".java", ".xml", ".yml", ".text", ".md", ".gitignore", "justfile"]

Collect all changes:
- lines containing patterns
- part of file name containing patterns
- part of directory name containing patterns

In [ ]:
def collect_rename_actions(root, name, verb):
    path = os.path.join(root, name)
    for pattern in patterns:
        if re.search(pattern, name):
            new_name = re.sub(pattern, patterns[pattern], name)
            new_path = os.path.join(root, new_name)
            yield (verb, path, new_path)


def collect_file_patches(root, name):
    path = os.path.join(root, name)
    with open(path, "r") as f:
        for line, text in enumerate(f.readlines()):
            new_text = text
            for pattern in patterns:
                if re.search(pattern, new_text):
                    new_text = re.sub(pattern, patterns[pattern], new_text)
            if new_text != text:
                yield (line, new_text)


def collect_directory_actions(root, name):
    for action in collect_rename_actions(root, name, "rename_dir"):
        yield action


def collect_file_actions(root, name):
    for action in collect_rename_actions(root, name, "rename_file"):
        yield action

    if not any((name.endswith(x) for x in includes)):
        return

    patches = list(collect_file_patches(root, name))
    if len(patches) > 0:
        file_path = os.path.join(root, name)
        yield ("patch_file", file_path, patches)


def collect_all_actions(root_path):
    for root, dirs, files in os.walk(root_path):
        for idx, name in enumerate(dirs):
            if name in ignores:
                del dirs[idx]
            else:
                for action in collect_directory_actions(root, name):
                    yield action
        for name in files:
            for action in collect_file_actions(root, name):
                yield action


actions = list(collect_all_actions("/mnt/media/Projects/Any2Json"))
display(list(reversed(actions)))

Apply all changes

In [120]:
def apply_file_patches(path, patches):
    os.rename(path, path + ".old")
    with open(path + ".old", "r") as fs:
        buffer = fs.readlines()
        for line, text in patches:
            buffer[line] = text
        with open(path, "w") as fd:
            fd.writelines(buffer)


for action in reversed(actions):
    verb, path, data = action
    match verb:
        case "patch_file":
            patches = data
            apply_file_patches(path, patches)
        case "rename_file":
            new_path = data
            os.rename(path, new_path)

for action in reversed(actions):
    verb, path, data = action
    match verb:
        case "rename_dir":
            new_path = data
            os.rename(path, new_path)

Cleanup

In [121]:
for root, _dirs, files in os.walk("/mnt/media/Projects/Any2Json"):
    for name in files:
        if name.endswith(".old"):
            path = os.path.join(root, name)
            os.remove(path)